
# 5.Analysing the daily news of the chosen media organization.
### Dash python framework. Part 2

### For more information regarding the Dash interface, see the below link:
https://dash.plot.ly/

## The code written in the dash application:
- get the daily news from the chose media organization.
- classify the news using the model ([1_Classifier.ipynb](https://github.com/paquinho89/TFM/blob/master/1_Classifier.ipynb))
- plot the result in a bar graph which shows the number of news of ech class

### NOTE: when the code is runned a link will pop up at the end of the code. The link will take a while to appear.  Click in that link to go to the interface. The server will run in your computer
### NOTE 2 : the data set is used to train the model. Therefore, this file should be saved in a folder called data to run the code.

#### To run the code, the below packages shoud be installed.

In [ ]:
pip install plotly && pip install dash

In [ ]:
! pip install dash==0.28.5  # The core dash backend
! pip install dash-html-components==0.13.2  # HTML components
! pip install dash-core-components==0.36.0  # Supercharged components
! pip install dash-table==3.1.2  # Interactive DataTable component (new!)

In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from sklearn.externals import joblib
import plotly.graph_objs as go
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import matplotlib.pyplot as plt
from sklearn.externals import joblib
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from textblob import TextBlob
from collections import Counter
from newsapi import NewsApiClient
from datetime import date, timedelta
from sklearn import preprocessing
from collections import Counter

### Code for the Dash framework

In [5]:
file=pd.read_csv('./data/dataset_news.csv', sep=',')
newsapi = NewsApiClient(api_key='9fe0d6dd387c40bc8cb5fdec346f0bda')
news_sources = newsapi.get_sources()
yesterday = str(date.today()- timedelta(1))
today = str(date.today())

app = dash.Dash()

app.layout = html.Div(children=[
        html.H1(children='News Classifier for the daily news', style={'textAlign':'center'}),
    
        html.Div(children=[
                html.Label('What the chosen media organization is talking about?')], style={'textAlign': 'center'}),
                html.Div(id='space', style={'padding': 10}),
                dcc.Markdown('**Choose a media organization:**'),
                dcc.Dropdown(
                id='drop-down-media-source',
                options=[{'label': i, 'value': i} 
                for i in list(map(lambda x: x['id'], filter(lambda x: x['language']=='en', news_sources['sources'])))]
                ),
                html.Div(id='space1', style={'padding': 10}),
                html.Button('Submit', id='button'),
                html.Div(id='space2', style={'padding': 10}),
                dcc.Markdown('''**The below graph shows the number of news of each class from the chosen media organization:**
                NOTE: Once the user has pressed the buttom, the graph takes a while to appear. (35 seconds)'''),
                dcc.Graph(id='bar_graph',style={'overflowY': 'scroll', 'height': 500})
        ])

@app.callback(
    dash.dependencies.Output(component_id='bar_graph', component_property='figure'),
    [dash.dependencies.Input('button', 'n_clicks')],
    [dash.dependencies.State('drop-down-media-source', 'value')])


def get_news_today(n_clicks, source_media):
    all_articles = newsapi.get_everything(sources= source_media,
                                          from_param= yesterday,
                                          to= today,
                                          language='en')
    clean_text=[]
    for text in all_articles['articles']:
        #if the value of the ['content']key is a strig, it is putted in a new list (clean_text).
        if type(text['content'])==str:
            clean_text.append(text) 
    #-------------------------------------        
    
    le = preprocessing.LabelEncoder()
    le.fit(["business", "science and technology", "entertainment", "health"])

    x_axis=[]

    corpus_business=''
    corpus_sciencetechnology=''
    corpus_entertainment=''
    corpus_health=''
    #Training the model
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(file['TITLE'])
    encoder = LabelEncoder()
    y = encoder.fit_transform(file['CATEGORY'])
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    nb = MultinomialNB()
    classifier_model = nb.fit(X_train, y_train)
        
    for i in range(len(clean_text)):
        prediction = nb.predict(vectorizer.transform([clean_text[i]['content']]))
        
        x_axis.append(list(le.inverse_transform(prediction)))
        if list(le.inverse_transform(prediction)) == ['entertainment']:
            corpus_business= corpus_business + '' + clean_text[i]['content']
        if list(le.inverse_transform(prediction)) == ['science and technology']:
            corpus_sciencetechnology= corpus_sciencetechnology + '' + clean_text[i]['content']
        if list(le.inverse_transform(prediction)) == ['entertainment']:
            corpus_entertainment= corpus_entertainment + '' + clean_text[i]['content']
        if list(le.inverse_transform(prediction)) == ['health']:
            corpus_health= corpus_health + '' + clean_text[i]['content']

    corpus=[corpus_business,corpus_sciencetechnology,corpus_entertainment,corpus_health]

   
    x_axis_list=[]
    for i in range(len(x_axis)):
        x_axis_list.append(x_axis[i][0])
    x=Counter(x_axis_list)

    labels, values = zip(*Counter(x_axis_list).items())
    
    indexes = np.array([label for label in labels])
    
    return {'data': [go.Bar(
            x= indexes,
            y= values)],
            'layout': go.Layout(
            xaxis=dict(tickangle=-15))}

app.css.append_css({'external_url': 'https://codepen.io/chriddyp/pen/bWLwgP.css'})

if __name__ == '__main__':
    app.run_server(debug=False) 
    

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [18/Nov/2018 16:44:57] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Nov/2018 16:45:00] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [18/Nov/2018 16:45:00] "GET /_dash-dependencies HTTP/1.1" 200 -
[2018-11-18 16:45:06,879] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/home/dsc/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1982, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/dsc/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1614, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/dsc/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1517, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/home/dsc/anaconda3/lib/python3.6/site-packages/flask/_compat.py", line 33, in reraise
    raise value
  File "/home/dsc/anaconda3/lib/python3.6/site-packages/flask/app.py", l